In [ ]:

class Historico:
    def __init__(self):
        self.transacoes = []

    def adicionar_transacao(self, transacao):
        self.transacoes.append(transacao)


from abc import ABC, abstractmethod

class Transacao(ABC):
    @abstractmethod
    def registrar(self, conta):
        pass

class Saque(Transacao):
    def __init__(self, valor):
        self.valor = valor

    def registrar(self, conta):
        if conta.sacar(self.valor):
            conta.historico.adicionar_transacao(f"Saque: R$ {self.valor:.2f}")
            return True
        return False


class Deposito(Transacao):
    def __init__(self, valor):
        self.valor = valor

    def registrar(self, conta):
        if conta.depositar(self.valor):
            conta.historico.adicionar_transacao(f"Depósito: R$ {self.valor:.2f}")
            return True
        return False

class Conta:
    def __init__(self, cliente, numero, agencia="0001"):
        self.saldo = 0.0
        self.numero = numero
        self.agencia = agencia
        self.cliente = cliente
        self.historico = Historico()
        cliente.adicionar_conta(self)

    def saldo(self):
        return self.saldo

    def sacar(self, valor):
        if valor > 0 and valor <= self.saldo:
            self.saldo -= valor
            return True
        return False

    def depositar(self, valor):
        if valor > 0:
            self.saldo += valor
            return True
        return False

class ContaCorrente(Conta):
    def __init__(self, cliente, numero, limite=1000, limite_saques=3):
        super().__init__(cliente, numero)
        self.limite = limite
        self.limite_saques = limite_saques
        self.saques_realizados = 0

    def sacar(self, valor):
        if self.saques_realizados < self.limite_saques and valor <= self.limite:
            if super().sacar(valor):
                self.saques_realizados += 1
                return True
        return False

class Cliente:
    def __init__(self, endereco):
        self.endereco = endereco
        self.contas = []

    def realizar_transacao(self, conta, transacao):
        return transacao.registrar(conta)

    def adicionar_conta(self, conta):
        self.contas.append(conta)

class PessoaFisica(Cliente):
    def __init__(self, cpf, nome, data_nascimento, endereco):
        super().__init__(endereco)
        self.cpf = cpf
        self.nome = nome
        self.data_nascimento = data_nascimento

cliente1 = PessoaFisica("123.456.789-00", "João Silva", "01/01/1980", "Rua A, 100")

conta1 = ContaCorrente(cliente1, numero=1234, limite=500, limite_saques=2)

cliente1.realizar_transacao(conta1, Deposito(200))
cliente1.realizar_transacao(conta1, Saque(50))
cliente1.realizar_transacao(conta1, Saque(30))

print(f"Saldo atual: R$ {conta1.saldo:.2f}")
print("Histórico de transações:")
for transacao in conta1.historico.transacoes:
    print(transacao)

Saldo atual: R$ 120.00
Histórico de transações:
Depósito: R$ 200.00
Saque: R$ 50.00
Saque: R$ 30.00
